# этап 1 
## нахождение на видео человека



In [54]:
!pip install ultralytics
!pip install clearml
!pip install supervision


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [56]:
from ultralytics import YOLO
import cv2
import numpy as np
import supervision as sv
import os

In [57]:
model = YOLO("yolov8n-pose.pt")

In [58]:
image = cv2.imread("data/push_up1.jpg")
results = model(source="data/right.jpg", show=False, conf=0.3, save=True)
results = model(source="data/push_up1.jpg", show=False, conf=0.3, save=True)

result = results[0]
results


image 1/1 /data/notebook_files/data/right.jpg: 640x512 1 person, 101.8ms
Speed: 3.5ms preprocess, 101.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 512)
Results saved to runs/pose/predict4

image 1/1 /data/notebook_files/data/push_up1.jpg: 640x288 1 person, 63.6ms
Speed: 2.0ms preprocess, 63.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 288)
Results saved to runs/pose/predict4


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: ultralytics.engine.results.Keypoints object
masks: None
names: {0: 'person'}
obb: None
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]]

In [58]:
def calculate_distance(keypoints, idx1, idx2):
    x1, y1 = keypoints.xy[0][idx1]
    x2, y2 = keypoints.xy[0][idx2]
    if keypoints.has_visible:
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        return distance
    else:
        return None


def choose_side(keypoints):
    r_shoulder = calculate_distance(keypoints, 6, 8)
    l_shoulder = calculate_distance(keypoints, 5, 7)
    return r_shoulder > l_shoulder

In [59]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle 

In [60]:
def get_push_up_angle(keypoints):
    if side:
        angle = calculate_angle(keypoints.xy[0][6],
                                keypoints.xy[0][8],
                                keypoints.xy[0][12], )
    else:
        angle = calculate_angle(keypoints.xy[0][5],
                                keypoints.xy[0][7],
                                keypoints.xy[0][11], )
    return angle



    

In [96]:
def check_position_body_line(keypoints):
    if side:
        #распремлена поясница
        angle1 =  calculate_angle(keypoints.xy[0][6],
                                keypoints.xy[0][12],
                                keypoints.xy[0][14],)
        #распрямлены ноги
        angle2 =  calculate_angle(keypoints.xy[0][12],
                                keypoints.xy[0][14],
                                keypoints.xy[0][16],)
        #изгиб локтя
        angle3 = calculate_angle(keypoints.xy[0][6],
                                keypoints.xy[0][8],
                                keypoints.xy[0][10],)
    else:
        angle1 =  calculate_angle(keypoints.xy[0][5],
                                keypoints.xy[0][11],
                                keypoints.xy[0][13],)
        angle2 =  calculate_angle(keypoints.xy[0][11],
                                keypoints.xy[0][13],
                                keypoints.xy[0][15],)
        angle3 = calculate_angle(keypoints.xy[0][5],
                                keypoints.xy[0][7],
                                keypoints.xy[0][9],)
    
    return angle1>120 and (angle2>120 or angle2<45) and angle3<120


def check_push_up_down(keypoints, prev_angle):
    angle = get_push_up_angle(keypoints=result.keypoints)
    if angle >= prev_angle:
        # prev_angle = max(angle, prev_angle)
        return True
    else:

        return False


def check_push_up(keypoints, prev_angle):
    angle = get_push_up_angle(keypoints=result.keypoints)
    if angle < prev_angle:
        return True
    else:
        return False


<!-- ![yolo keypoints](/data/keypoints_yolo.png "yolo keypoints") -->
<image src="/data/keypoints_yolo.png" alt="yolo keypoints">

In [89]:
def print_count(image, pull_ups, push_ups, squats, person):
    org = [20, 100]  # Например, (x, y) = (50, 50)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 4  # Масштаб шрифта
    font_color = (0, 200, 0)  # Цвет шрифта в формате BGR   
    thickness = 4  # Толщина шрифта
    org[1] = org[1] + (person - 1) * 400
    # text = "person: {0} ".format(person)
    # cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    # org[1] += 100
    text = "pull ups: {0} ".format(pull_ups)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "push ups: {0} ".format(push_ups)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "squats: {0}".format(squats)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)


def print_debag_info(image,  movment_down, prev_angle,position_line):
    org = [20, 100]  # Например, (x, y) = (50, 50)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3  # Масштаб шрифта
    font_color = (0, 200, 0)  # Цвет шрифта в формате BGR   
    thickness = 3  # Толщина шрифта
    org[1] = org[1] + 400
    # text = "{0} ".format(push_ups)
    # cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    # org[1] += 100
    text = "movment_down: {0} ".format(movment_down)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "prev_angle: {0} ".format(prev_angle)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "pos= line{0}".format(position_line)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)

# image1 = cv2.imread("data/push_up1.jpg")
# print_count(image1,1,1,1)
# cv2.imwrite('cropped_image.jpg', image1)

In [99]:
video = cv2.VideoCapture("data/push_ups1.mp4")
# output_path = os.path.join("", "output2.mp4")
video_info = sv.VideoInfo.from_video_path("data/push_ups1.mp4")
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter("output3.mp4", fourcc, video_info.fps ,
                               (video_info.width, video_info.height))
# result = model(source="data/push_ups1.mp4", show=False, conf=0.3, save=True)
push_up_count = 0
pull_up_count = 0
squats_count = 0
movment_down = True
prev_angle = get_push_up_angle(keypoints=result.keypoints)
cadr_num = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    if cadr_num % 5 == 0:
        results = model.predict(frame)
        person = 1
        # for result in results:
        result = results[0]

        side = choose_side(keypoints=result.keypoints)
        
        if check_position_body_line(result.keypoints):
            # push_ups подсчёт
            if movment_down and check_push_up_down(result.keypoints, prev_angle):
                movment_down = False
            # else:
            # prev_angle = max(get_push_up_angle(keypoints=result.keypoints), prev_angle)
            if not movment_down and check_push_up(result.keypoints, prev_angle):
                push_up_count += 1
                movment_down = True
            prev_angle = get_push_up_angle(result.keypoints)
            # pull+ups подчё

        # print_count(frame, pull_up_count, push_up_count, squats_count, person)
        # print_debag_info(frame, movment_down, prev_angle,check_position_body_line(result.keypoints))
        # output_video.write(frame)
    cadr_num += 1
    print_count(frame, pull_up_count, push_up_count, squats_count, person)
    output_video.write(frame)
    # for box in result.boxes:
    #     class_id = result.names[box.cls[0].item()]
    #     cords = box.xyxy[0].tolist()
    #     cords = [round(x) for x in cords]
    #     conf = round(box.conf[0].item(), 2)
    #     cv2.rectangle(frame, [cords[0],cords[1]], [cords[2],cords[3]], (0, 255, 0), 2)

    # print("Object type:", class_id)
    # print("Coordinates:", cords)
    # print("Probability:", conf)
    # print("---")    

video.release()
output_video.release()
cv2.destroyAllWindows()


0: 640x384 1 person, 168.6ms
Speed: 2.7ms preprocess, 168.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 85.0ms
Speed: 2.5ms preprocess, 85.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 87.6ms
Speed: 2.5ms preprocess, 87.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 78.9ms
Speed: 5.0ms preprocess, 78.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 79.5ms
Speed: 2.6ms preprocess, 79.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 91.2ms
Speed: 2.5ms preprocess, 91.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 79.2ms
Speed: 7.7ms preprocess, 79.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 83.0ms
Speed: 7.8ms preprocess, 83.0ms inference, 1.0ms postprocess per image at shape (1, 3, 